In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from environment import Environment

Step 1：Prepare data (batch)

Step 2：Neural Network 

Step 3: Define loss function and optimizer

Step 4：Train the network

Step 5：GPU

### Step 1. Prepare Data
* Tensor
* Shape

In [2]:
# Prepare Data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)

print('original dtype:',type(trainset.data))
print('original shape:',trainset.data.shape)

Files already downloaded and verified
original dtype: <class 'numpy.ndarray'>
original shape: (50000, 32, 32, 3)


In [3]:
def batch(data,batch_size):
    minibatch = random.sample(data, batch_size)
    minibatch = np.array(minibatch).transpose(0,3,1,2)
    minibatch = torch.tensor(minibatch/ 255.0)
    return minibatch

In [4]:
minibatch = batch(list(trainset.data),32)
print(minibatch.shape)
print(minibatch.dtype)

torch.Size([32, 3, 32, 32])
torch.float64


In [5]:
# what does env.reset() look like?
env = Environment('BreakoutNoFrameskip-v4', "", atari_wrapper=False, test=False)
frame = env.reset()
print("shape:",frame.shape)
print("dtype:",frame.dtype)

shape: (210, 160, 3)
dtype: uint8


/home/hren/anaconda3/envs/pytorch/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [6]:
# what does env.reset() look like?
env = Environment('BreakoutNoFrameskip-v4', "", atari_wrapper=True, test=False)
frame = env.reset()
print("shape:",frame.shape)
print("dtype:",frame.dtype) 
# convert to float and between 0 and 1 before training
# convert to tensor, shape as [batch_size, channel, H, W]

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
WARN: <class 'atari_wrapper.FrameStack'> doesn't implement 'reset' method, but it implements deprecated '_reset' method.
shape: (84, 84, 4)
dtype: uint8


### Step 2. Neural Network
* init()
  * layers
* forward
  * activation function + combine layers

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        """
        Define layers
        """
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """
        Add activation function
        Combine layers to a Neural Network
        """
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
net = Net()

### Step 3. Define loss and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Step 4. Train the network

In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified


In [11]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        ####################################################
        # tensor
        # batch_size, channel, H, W
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        ####################################################
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.210
[1,  4000] loss: 1.904
[1,  6000] loss: 1.731
[1,  8000] loss: 1.602
[1, 10000] loss: 1.553
[1, 12000] loss: 1.517
[2,  2000] loss: 1.467
[2,  4000] loss: 1.414
[2,  6000] loss: 1.384
[2,  8000] loss: 1.373
[2, 10000] loss: 1.372
[2, 12000] loss: 1.344
Finished Training


### Step 5. GPU

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [13]:
# Assign network to cuda
net = Net().to(device)

In [14]:


# Assign input and output to cuda
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        ####################################################
        # tensor
        # batch_size, channel, H, W
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        ####################################################
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.305
[1,  4000] loss: 2.305
[1,  6000] loss: 2.306
[1,  8000] loss: 2.305
[1, 10000] loss: 2.305
[1, 12000] loss: 2.305
[2,  2000] loss: 2.305
[2,  4000] loss: 2.305
[2,  6000] loss: 2.305
[2,  8000] loss: 2.305
[2, 10000] loss: 2.305
[2, 12000] loss: 2.307
Finished Training
